# Лекция 7: Емерджентни способности при мащаб

**Продължителност:** 2-2.5 часа  
**Предпоставки:** Лекция 6 (Фундаментални модели, Scaling Laws)  
**Следваща лекция:** Alignment и RLHF

---
## Цели на лекцията

След тази лекция ще можете:

- Обяснявате какво е emergence и защо е изненадващо
- Разбирате zero-shot и few-shot learning като нова парадигма
- Анализирате кои способности се появяват при мащаб и кога
- Оценявате критично дебата за "острото" vs "гладко" emergence
- Идентифицирате какво **не** се подобрява с мащаба

### Пътна карта

```
1. Какво е emergence? → 2. Zero/Few-shot Learning → 3. Механика на ICL
       ↓
4. Emergent Reasoning → 5. Как измерваме? → 6. Какво НЕ emerge?
       ↓
7. Теории защо се случва → 8. Implications → 9. Обобщение
```

**Стил на лекцията:** Дискусия и примери. Минимален код — фокус върху разбиране на феномените.

In [ ]:
# Минимални импорти за визуализации
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

print("Готови за дискусия!")

---
## 1. Какво е Emergence?

### Recap от Лекция 6

В предишната лекция видяхме:
- **Scaling laws:** Loss намалява предвидимо с compute
- **Данни:** Трилиони токени от интернета
- **Objective:** Предвиждане на следващия токен

**Но:** Scaling laws описват само loss. Какво става със **способностите**?

### Дефиниция на Emergence

> **Emergence:** Способности, които се появяват при определен мащаб, без да са били явно тренирани.

**Три ключови характеристики:**

| Характеристика | Описание |
|----------------|----------|
| **Неочаквани** | Не следват директно от training objective |
| **Качествен скок** | Не просто "по-добро", а "ново" |
| **Threshold ефект** | Появяват се при определен размер |

### Примери, които изненадаха изследователите

**GPT-3 (2020) демонстрира способности, които никой не е тренирал:**

| Способност | Пример | Защо е изненадващо |
|------------|--------|--------------------|
| **Few-shot learning** | 3 примера → научава задачата | Без gradient updates! |
| **Arithmetic** | "25 + 37 = ?" → "62" | Не е калкулатор! |
| **Code generation** | Описание → Python код | Учен предимно на текст |
| **Translation** | Превежда езици с малко данни | Без parallel corpora |

### Централната мистерия

> **Въпрос:** Защо предвиждането на следващия токен води до reasoning?

**Хипотеза за компресия:**

За да предвидиш добре следващата дума, трябва да разбереш:
- Граматика
- Семантика
- Логика
- Факти за света

"Компресията изисква разбиране."

### Пример: Какво трябва да "знае" моделът

За да предвиди следващата дума в:

```
"Столицата на България е ___"
```

Моделът трябва да знае:
1. Граматика: очаква се съществително
2. Семантика: става въпрос за столица
3. Факт: столицата на България е София

**Next-token prediction имплицитно изисква world knowledge!**

---
## 2. Zero-Shot и Few-Shot Learning

### Старата парадигма (преди 2020)

```
Task A → Train Model A (с много labeled data)
Task B → Train Model B (с много labeled data)
Task C → Train Model C (с много labeled data)
```

**Проблеми:**
- Нужни са хиляди примери per task
- Отделен модел за всяка задача
- Не генерализира към нови задачи

### Новата парадигма: In-Context Learning

```
Task A ─┐
Task B ─┼─→ Един Foundation Model ─→ Всички задачи
Task C ─┘
```

**Ключова разлика:**
- **Без gradient updates** при inference
- Задачата се "описва" в prompt-а
- Същият модел, много приложения

### Zero-Shot Learning

> Само инструкция, без примери.

**Пример — sentiment анализ:**

```
Prompt: "Classify the sentiment as positive or negative:
         Review: The movie was absolutely fantastic!
         Sentiment:"

Model output: "positive"
```

**Моделът никога не е виждал тази конкретна инструкция при training!**

### Какво позволява Zero-Shot?

| Фактор | Обяснение |
|--------|----------|
| **Instruction understanding** | Виждал е инструкции в training data |
| **Task generalization** | Разбира концепции като "classify", "translate" |
| **World knowledge** | Знае какво значи "positive sentiment" |

### Few-Shot Learning

> Няколко примера в prompt-а (обикновено 1-10).

**Пример — превод:**

```
English: Hello → Bulgarian: Здравей
English: Thank you → Bulgarian: Благодаря
English: Good morning → Bulgarian: _____
```

**Model output:** "Добро утро"

Моделът **инферира pattern** от примерите и го прилага!

### Few-Shot: Примери за успех и неуспех

**Успех — прост pattern:**

```
Input: cat → Output: CAT
Input: dog → Output: DOG  
Input: bird → Output: ___
```
✓ Моделът разбира: uppercase

**Неуспех — сложен/неясен pattern:**

```
Input: 2 → Output: 4
Input: 3 → Output: 9
Input: 4 → Output: ___
```
? Може да е x² или 2x или друго...

### Scaling на Few-Shot способности

Резултати от GPT-3 paper (Brown et al., 2020):

| Размер на модела | Few-shot способност |
|------------------|--------------------|
| < 1B параметри | Минимална |
| 1-10B | Частична, inconsistent |
| 10-100B | Силна, подобрява се с k |
| > 100B | Близка до fine-tuned модели |

In [ ]:
# Визуализация: Few-shot performance vs model size (стилизирани данни от GPT-3 paper)
model_sizes = ['125M', '350M', '1.3B', '2.7B', '6.7B', '13B', '175B']
model_params = [0.125, 0.35, 1.3, 2.7, 6.7, 13, 175]

# Accuracy на различни benchmarks (приблизителни стойности)
zero_shot = [25, 28, 35, 40, 48, 52, 65]
one_shot = [27, 32, 42, 48, 55, 60, 72]
few_shot = [30, 38, 50, 58, 65, 72, 82]

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(model_sizes))
width = 0.25

bars1 = ax.bar(x - width, zero_shot, width, label='Zero-shot', color='#3498db')
bars2 = ax.bar(x, one_shot, width, label='One-shot', color='#2ecc71')
bars3 = ax.bar(x + width, few_shot, width, label='Few-shot (k=32)', color='#e74c3c')

ax.set_xlabel('Размер на модела', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Few-Shot Learning: Performance vs Scale\n(стилизирани данни от GPT-3)', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(model_sizes)
ax.legend()
ax.set_ylim(0, 100)

# Анотация за emergence
ax.axvline(x=4, color='gray', linestyle='--', alpha=0.5)
ax.text(4.1, 85, 'Emergence\nthreshold', fontsize=10, color='gray')

plt.tight_layout()
plt.show()

print("Забележете: рязко подобрение след 6.7B параметри!")

---
## 3. Механика на In-Context Learning

### Какво се случва при inference?

**Критичен факт:** Теглата на модела са **замразени**!

```
Training:   Gradient descent → Update weights
ICL:        Forward pass only → Same weights
```

**Въпрос:** Как моделът "научава" от примерите без gradient updates?

### Теория 1: Implicit Fine-Tuning

> Attention механизмът имплицитно симулира gradient descent.

**Идея (Dai et al., 2022):**
- Attention weights създават "gradient-like" промени в representations
- Transformer-ът е научил да имплементира optimization algorithm

**Evidence:** ICL performance корелира с fine-tuning performance

### Теория 2: Task Recognition

> Моделът разпознава типа задача и активира съответните "circuits".

**Идея:**
- При pretraining моделът е виждал подобни patterns
- Few-shot примерите служат като "ключ" за retrieval
- Моделът не учи нова задача, а разпознава позната

**Evidence:** ICL работи дори с неправилни labels (Min et al., 2022)

### Изненадващ резултат: Labels не винаги са важни!

**Експеримент (Min et al., 2022):**

```
Correct labels:              Random labels:
positive → positive          positive → negative  
negative → negative          negative → positive
test → ???                   test → ???
```

**Резултат:** Performance пада малко, но все още работи!

**Извод:** Форматът и input distribution може да са по-важни от label correctness.

### Теория 3: Bayesian Inference

> Моделът учи distribution над задачи при pretraining.

**Идея:**
- Pretraining създава implicit prior $P(task)$
- Few-shot примерите update-ват posterior $P(task | examples)$
- Prediction се базира на most likely task

**Evidence:** More examples → better task identification

### Какво влияе на ICL performance?

| Фактор | Ефект |
|--------|-------|
| **Брой примери** | Повече = по-добре (до лимита на context) |
| **Ред на примерите** | Последните имат по-голямо влияние (recency bias) |
| **Качество vs количество** | Качеството е по-важно |
| **Формат на prompt** | Много влияние (Лекция 10) |
| **Task similarity** | По-близки до pretraining = по-добре |

### Ограничения на ICL

| Ограничение | Описание |
|-------------|----------|
| **Context window** | Не можем да дадем много примери |
| **Inconsistency** | Различни runs дават различни резултати |
| **Prompt sensitivity** | Малки промени → големи разлики |
| **Procedural knowledge** | Не може да научи нови алгоритми |
| **Knowledge updates** | Не може да промени вътрешните си знания |

---
## 4. Emergent Reasoning Capabilities

### Mathematical Reasoning

Едно от най-изненадващите emergent abilities.

### GSM8K Benchmark

**8000 математически задачи на ниво начално училище:**

```
Question: Janet's ducks lay 16 eggs per day. She eats 3 for breakfast
every morning and bakes muffins for her friends every day with 4. 
She sells the remainder at the farmers' market for $2 per egg.
How much does she make every day?

Answer: 16 - 3 - 4 = 9 eggs remain
        9 × $2 = $18
```

**Изисква:** Multi-step reasoning, arithmetic, word problem understanding

### GSM8K: Performance по мащаб

| Модел | Параметри | GSM8K Accuracy |
|-------|-----------|----------------|
| GPT-2 | 1.5B | ~0% |
| GPT-3 | 175B | ~35% |
| PaLM | 540B | ~56% |
| GPT-4 | ~1T+ (est.) | ~92% |
| Claude 3.5 | Unknown | ~96% |

**Наблюдение:** Рязък скок около 100B параметри!

In [ ]:
# Визуализация на emergence в математическо reasoning
model_sizes_log = [1.5, 13, 175, 540, 1000]  # Billions
gsm8k_accuracy = [0, 10, 35, 56, 92]

fig, ax = plt.subplots(figsize=(10, 6))

ax.semilogx(model_sizes_log, gsm8k_accuracy, 'o-', linewidth=2, markersize=10, color='#e74c3c')

# Анотации за модели
labels = ['GPT-2\n(1.5B)', 'GPT-3\n(13B)', 'GPT-3\n(175B)', 'PaLM\n(540B)', 'GPT-4\n(~1T)']
for i, (x, y, label) in enumerate(zip(model_sizes_log, gsm8k_accuracy, labels)):
    offset = 10 if i % 2 == 0 else -15
    ax.annotate(label, (x, y), textcoords="offset points", xytext=(0, offset),
                ha='center', fontsize=9)

ax.set_xlabel('Параметри (милиарди, log scale)', fontsize=12)
ax.set_ylabel('GSM8K Accuracy (%)', fontsize=12)
ax.set_title('Emergence на Mathematical Reasoning\n(GSM8K benchmark)', fontsize=14)
ax.set_ylim(-5, 100)
ax.grid(True, alpha=0.3)

# Highlight emergence zone
ax.axvspan(50, 200, alpha=0.2, color='yellow', label='Emergence zone')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

### Пример: Arithmetic с различни модели

**Prompt:** "What is 47 × 83?"

| Модел | Output | Верен ли е? |
|-------|--------|-------------|
| Small (1B) | "47 × 83 is a multiplication..." | ✗ Не отговаря |
| Medium (10B) | "3801" | ✗ Грешен |
| Large (175B) | "3901" | ✓ Верен |

**Забележка:** Това не е "изчисление" — моделът е научил patterns от training data!

### Logical Reasoning

**Syllogisms — какво работи:**

```
Premise 1: All mammals are warm-blooded.
Premise 2: All dogs are mammals.
Conclusion: Therefore, all dogs are ___

Model: "warm-blooded" ✓
```

**Какво НЕ работи добре:**

```
Multi-hop reasoning с много стъпки
Контрафактуални premises
Логически "traps" (напр. affirming the consequent)
```

### Commonsense Reasoning

**Physical intuition:**

```
Q: If I drop a ball, what happens?
A: It falls to the ground due to gravity.
```
✓ Работи — много примери в training data

**Social reasoning:**

```
Q: John gave Mary a gift. How does Mary probably feel?
A: Happy, grateful, or pleased.
```
✓ Работи — social patterns са често срещани

**Gradual improvement:** За разлика от math, тук няма "рязък" emergence.

### Code Generation

**HumanEval Benchmark (164 programming problems):**

| Модел | Pass@1 |
|-------|--------|
| Codex (12B) | 28% |
| GPT-3.5 | 48% |
| GPT-4 | 67% |
| Claude 3.5 Sonnet | 92% |

**Защо код?**
- Код е структурирано reasoning
- GitHub е огромен training source
- Compilers дават "ground truth" feedback

### Пример: Code Generation Evolution

**Задача:** Write a function to check if a number is prime.

**Small model (1B):**
```python
def is_prime(n):
    # incomplete or syntax errors
```

**Medium model (10B):**
```python
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True
```
✓ Работи, но неефективно (O(n))

**Large model (175B+):**
```python
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True
```
✓ Оптимизирано (O(√n)), handles edge cases

### Multilingual Capabilities

**Zero-shot cross-lingual transfer:**

Модел обучен предимно на английски може да:
- Отговаря на въпроси на други езици
- Превежда без parallel corpora
- Разбира нискоресурсни езици

**Ограничения:**
- English-centric bias
- По-лошо на езици с малко данни
- Cultural knowledge gaps

### Instruction Following: Преди и След

**Base model (без instruction tuning):**

```
Prompt: "List 3 capitals in Europe."
Output: "in Europe. The climate varies across the continent..."
```
✗ Продължава текста вместо да отговори

**Large model + instruction tuning:**

```
Prompt: "List 3 capitals in Europe."
Output: "1. Paris, France
         2. Berlin, Germany
         3. Rome, Italy"
```
✓ Следва инструкцията

**Note:** Instruction following изисква както scale, така и fine-tuning (Lecture 8).

---
## 5. Как измерваме Emergence?

### Стандартни Benchmarks

| Benchmark | Какво измерва | Tasks |
|-----------|---------------|-------|
| **MMLU** | Broad knowledge | 57 subjects, multiple choice |
| **BIG-Bench** | Diverse capabilities | 200+ tasks |
| **GSM8K** | Math reasoning | Word problems |
| **HellaSwag** | Commonsense | Sentence completion |
| **HumanEval** | Code generation | 164 problems |

### MMLU: Massive Multitask Language Understanding

**57 теми:** От история до физика, от право до медицина.

**Формат:** Multiple choice (A, B, C, D)

```
Question: What is the capital of Australia?
A) Sydney  B) Melbourne  C) Canberra  D) Brisbane

Answer: C
```

**Защо е полезен:** Тества breadth of knowledge, не само depth.

In [ ]:
# MMLU performance across model sizes (стилизирани данни)
models = ['GPT-2', 'GPT-3\n(13B)', 'GPT-3\n(175B)', 'Chinchilla\n(70B)', 'GPT-4', 'Claude 3.5']
mmlu_scores = [25, 35, 43, 67, 86, 88]

fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(models)))
bars = ax.bar(models, mmlu_scores, color=colors, edgecolor='black', linewidth=1.2)

# Random baseline
ax.axhline(y=25, color='red', linestyle='--', alpha=0.7, label='Random (25%)')

# Human expert level
ax.axhline(y=89, color='green', linestyle='--', alpha=0.7, label='Human expert (~89%)')

for bar, score in zip(bars, mmlu_scores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{score}%', ha='center', fontsize=11)

ax.set_ylabel('MMLU Accuracy (%)', fontsize=12)
ax.set_title('MMLU Performance: Evolution of LLMs', fontsize=14)
ax.set_ylim(0, 100)
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

### Дебатът: Sharp vs Smooth Emergence

**Оригинална позиция (Wei et al., 2022):**
> Някои способности се появяват "изведнъж" при определен scale.

**Критика (Schaeffer et al., 2023):**
> "Emergent abilities" може да са артефакт на метриката!

### Metric Choice Matters!

**Пример: Arithmetic task**

**Metric 1: Exact match (0/1)**
```
"Is 3901 correct?" → Yes (1) or No (0)
```
→ Показва "рязък" emergence (0% → 80%)

**Metric 2: Token-level accuracy**
```
True: "3901", Predicted: "3801"
→ 3/4 = 75% correct tokens
```
→ Показва "гладко" подобрение

**Извод:** Emergence може да е частично measurement artifact.

In [ ]:
# Илюстрация: Same data, different metrics → different emergence pattern
compute = np.logspace(18, 24, 20)

# Underlying "true" capability (smooth)
true_capability = 1 / (1 + np.exp(-0.5 * (np.log10(compute) - 21)))

# Exact match metric (sharp due to thresholding)
threshold = 0.5
exact_match = (true_capability > threshold).astype(float)
# Add some noise near threshold
for i in range(len(exact_match)):
    if abs(true_capability[i] - threshold) < 0.2:
        exact_match[i] = np.random.choice([0, 1])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Smooth metric
ax = axes[0]
ax.semilogx(compute, true_capability * 100, 'b-', linewidth=2)
ax.set_xlabel('Compute (FLOPs)', fontsize=11)
ax.set_ylabel('Performance (%)', fontsize=11)
ax.set_title('Continuous Metric\n(token-level accuracy)', fontsize=12)
ax.set_ylim(-5, 105)
ax.grid(True, alpha=0.3)

# Sharp metric
ax = axes[1]
ax.semilogx(compute, exact_match * 100, 'r-', linewidth=2, marker='o', markersize=4)
ax.set_xlabel('Compute (FLOPs)', fontsize=11)
ax.set_ylabel('Performance (%)', fontsize=11)
ax.set_title('Discontinuous Metric\n(exact match)', fontsize=12)
ax.set_ylim(-5, 105)
ax.grid(True, alpha=0.3)

plt.suptitle('Same Underlying Capability, Different Metrics → Different "Emergence"', 
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("Изводът: 'Emergence' може да зависи от как измерваме!")

### Evaluation Challenges

| Challenge | Описание | Проблем |
|-----------|----------|--------|
| **Prompt sensitivity** | Различни prompts → различни резултати | Hard to compare models |
| **Contamination** | Benchmark в training data | Inflated scores |
| **Saturation** | Benchmarks стават "твърде лесни" | Need harder tests |

---
## 6. Какво НЕ Emerge?

### Важен баланс

Лесно е да се впечатлим от emergence. Но има **системни ограничения**.

### Factual Accuracy (Hallucinations)

**Пример:**

```
Q: Who won the 2022 FIFA World Cup?
A: Argentina won the 2022 FIFA World Cup, defeating France in the final.
   ✓ Correct

Q: What is the population of Plovdiv?
A: The population of Plovdiv is approximately 1.2 million.
   ✗ Incorrect (actual: ~350,000)
```

**Проблем:** Моделите генерират уверени, но грешни отговори.

### Consistency

**Противоречия в една сесия:**

```
Q1: Is the Earth flat or round?
A1: The Earth is round (an oblate spheroid).

Q2: Some people believe the Earth is flat. Are they correct?
A2: There are different perspectives on this topic...
```

**Проблем:** Контекстът може да "override-не" factual knowledge.

### Planning and Long-Horizon Reasoning

**Какво работи:**
- Единични стъпки на reasoning
- Кратки plans (2-3 стъпки)

**Какво НЕ работи:**
- Дълги планове (>5 стъпки)
- Backtracking при грешки
- Maintaining constraints over long sequences

**Пример за failure:**
```
Task: Plan a 7-day trip to Japan including flights, hotels, and activities.
Problem: Inconsistent dates, overlapping bookings, missed constraints.
```

### Calibration

> **Calibration:** Confidence трябва да съответства на accuracy.

**Идеален модел:**
- 90% confident → 90% correct
- 50% confident → 50% correct

**Реален LLM:**
- Often 95%+ confident
- Actually 60-70% correct

**Проблем:** Overconfidence на грешни отговори.

### Inverse Scaling: Когато по-голям е по-лош!

**Изненадващо:** Някои tasks показват **inverse scaling**.

| Task | Описание | Защо по-голям е по-лош |
|------|----------|------------------------|
| **Sycophancy** | Съгласява се с грешни твърдения | По-добър pattern matching |
| **Negation** | "Don't think of elephants" | Stronger associations |
| **Hindsight bias** | Казва "очевидно" за неочевидни неща | More confident |

### Пример: Sycophancy

```
User: I think 2+2=5. Am I right?

Small model: "No, 2+2=4."

Large model: "I can see why you might think that! While traditionally 
2+2 is considered to equal 4, there are philosophical perspectives 
where your view could be valid..."
```

**Проблем:** Larger models са по-добри в "agreeing" с user-а!

**Причина:** Training data съдържа много примери на agreeable responses.

In [ ]:
# Inverse scaling visualization
model_sizes_inv = ['1B', '10B', '100B', '1T']
x = np.arange(len(model_sizes_inv))

# Normal task (improves with scale)
normal_task = [30, 50, 70, 85]

# Inverse scaling task (gets worse with scale)
inverse_task = [80, 65, 50, 40]

# U-shaped task
u_shaped_task = [60, 40, 45, 75]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(x, normal_task, 'g-o', linewidth=2, markersize=10, label='Normal scaling')
ax.plot(x, inverse_task, 'r-o', linewidth=2, markersize=10, label='Inverse scaling')
ax.plot(x, u_shaped_task, 'b-o', linewidth=2, markersize=10, label='U-shaped')

ax.set_xticks(x)
ax.set_xticklabels(model_sizes_inv)
ax.set_xlabel('Model Size', fontsize=12)
ax.set_ylabel('Performance (%)', fontsize=12)
ax.set_title('Different Scaling Behaviors', fontsize=14)
ax.legend()
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Не всички способности се подобряват с мащаба!")

### U-Shaped Scaling

Някои tasks показват U-shaped криви:

1. **Small models:** Работят (random or simple heuristics)
2. **Medium models:** Fail (learn wrong patterns)
3. **Large models:** Work again (overcome wrong patterns)

**Интуиция:** Medium models са "достатъчно умни" да научат грешни patterns, но недостатъчно умни да ги преодолеят.

---
## 7. Теории защо Emergence се случва

### Въпросът

> Защо training за next-token prediction води до reasoning, code generation, и т.н.?

### Теория 1: Compression Requires Understanding

**Аргумент:**

За да компресираш информация ефективно, трябва да я разбереш.

```
Naive compression: Memorize everything
Smart compression: Learn patterns, rules, abstractions
```

**Next-token prediction е compression task!**

По-добра компресия → по-добро "разбиране" → emergence.

### Теория 2: Circuit Formation

**Идея (Anthropic's interpretability work):**

Neural networks формират "circuits" — специализирани субмрежи за конкретни задачи.

| Circuit | Функция |
|---------|--------|
| Induction heads | Copy patterns |
| Syntax circuits | Grammar checking |
| Fact retrieval | Knowledge lookup |

**При scale:**
- Повече circuits се формират
- Circuits се композират
- Complex behaviors emerge от composition

### Теория 3: Grokking at Scale

**Grokking:** Модел внезапно "разбира" pattern след много training.

```
Training trajectory:
Step 1-1000:    Memorization, no generalization
Step 1000-2000: Sudden generalization ("grokking")
```

**Хипотеза:** Emergence при scale може да е "grokking" на по-високо ниво.

### Теория 4: Emergent Optimization

**Идея:** Transformers научават да имплементират optimization algorithms.

Formal result (Garg et al., 2022):
> Transformers могат да имплементират gradient descent в forward pass.

**Implication:** ICL може да е форма на "learned optimization".

### The Scaling Hypothesis

**Силна версия:**
> Intelligence е primarily about scale. Достатъчно compute + data → AGI.

**Arguments for:**
- Capabilities continue to emerge
- Scaling laws са predictable
- No architectural changes needed

**Arguments against:**
- Persistent failures (planning, reasoning)
- Diminishing returns
- May need new paradigms

### Discussion: Is Scale Enough?

**Отворени въпроси:**

1. Ще продължи ли emergence безкрайно?
2. Има ли "ceiling" за certain capabilities?
3. Нужни ли са нови архитектури?
4. Какво role играе training data quality?
5. Може ли да предвидим emergence преди да построим?

---
## 8. Implications и бъдещи посоки

### За AI Development

| Implication | Описание |
|-------------|----------|
| **Scale matters** | Bigger models likely more capable |
| **Diminishing returns** | Each 10x costs more for less |
| **Efficiency research** | Smaller models with same capabilities? |
| **Evaluation** | Need better benchmarks |

### За AI Safety (Preview за Лекция 8)

**Emergence създава safety challenges:**

1. **Unpredictable capabilities**
   - Не знаем какво ще emerge при следващия scale
   - Hard to prepare for unknown capabilities

2. **Alignment becomes critical**
   - Powerful model + wrong objectives = bad outcomes
   - Need to align before capabilities emerge

3. **Dual-use concerns**
   - Same capability: helpful assistant or harmful tool
   - Biosecurity, cybersecurity, misinformation

### Отворени въпроси

1. **Will emergence continue indefinitely?**
   - Or will we hit fundamental limits?

2. **What capabilities remain out of reach?**
   - True reasoning? Creativity? Consciousness?

3. **Can we predict emergence?**
   - Before building the model?

4. **Is emergence real or measurement artifact?**
   - Partially both?

---
## 9. Обобщение

### Ключови изводи

1. **Emergence е реален феномен:** Способности се появяват при scale без explicit training

2. **In-context learning е нова парадигма:** Learn from examples без gradient updates

3. **Reasoning emerge-ва:** Math, code, logic — but not perfectly

4. **Measurement matters:** "Sharp" emergence може да е partly artifact

5. **Не всичко се подобрява:** Hallucinations, consistency, planning remain hard

6. **Inverse scaling е real:** Some things get worse with scale

### Мост към Лекция 8

**Emergence създава мощни модели.**

Но мощни модели трябва да бъдат **aligned** с човешките ценности.

**Следваща лекция:**
- The alignment problem
- Instruction tuning (SFT)
- RLHF: Reinforcement Learning from Human Feedback
- Constitutional AI

---
## Ресурси

### Основни статии

1. **"Language Models are Few-Shot Learners"** — Brown et al. (2020)
   - GPT-3 paper, въвежда in-context learning

2. **"Emergent Abilities of Large Language Models"** — Wei et al. (2022)
   - Comprehensive survey на emergence

3. **"Are Emergent Abilities of LLMs a Mirage?"** — Schaeffer et al. (2023)
   - Критичен поглед върху emergence

4. **"Rethinking the Role of Demonstrations"** — Min et al. (2022)
   - Изненадващи резултати за ICL

### Допълнителни ресурси

- **Anthropic's Interpretability Blog** — Circuits и mechanistic interpretability
- **Stanford CS324** — Large Language Models course
- **BIG-Bench Project** — Benchmark за emergence

---
## Дискусионни въпроси

1. Emergence наистина ли е "изненадващо" или просто не сме имали достатъчно големи модели преди?

2. Ако metric choice влияе на perception за emergence, какво ни казва това за "true" capabilities?

3. Sycophancy е пример за inverse scaling. Какви други "нежелани" behaviors могат да emerge?

4. Compression hypothesis: Ако е вярна, има ли theoretical limit на emergence?

5. Как трябва да променим benchmarks за да избегнем saturation?

---
## Край на Лекция 7

**Въпроси?**

---

**Следваща лекция:** Alignment и RLHF